# P7 : Détectez les Bad Buzz grâce au Deep Learning

## Traitement du contenu des tweets (pour modèle glove)

In [1]:
import os
import pandas as pd
df_dep = pd.read_csv('P7_03_fichiercsv_tweet.csv')
df_dep.head()

,label,date_du_poste,utilisateur,tweet
0,0,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...
1,0,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire
3,0,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,Mon Apr 06 22:20:00 PDT 2009,joy_wolf,@Kwesidei not the whole crew


In [2]:
df_dep.label.value_counts()

4    800000
0    799999
Name: label, dtype: int64

In [3]:
df_dep['polarite'] = df_dep['label'].map({
    4 : 'positive',
    0 : 'negative'
})

In [4]:
df_dep.polarite.value_counts()

positive    800000
negative    799999
Name: polarite, dtype: int64

In [5]:
# Extrait des données
n_sample = 20000
df_neg= df_dep[df_dep['polarite'] == 'negative'].sample(n=n_sample)
df_pos = df_dep[df_dep['polarite'] == 'positive'].sample(n=n_sample)

data = pd.concat([df_neg, df_pos], ignore_index=True)

In [6]:
data.polarite.value_counts()

negative    20000
positive    20000
Name: polarite, dtype: int64

## Nettoyage des tweets

In [7]:
#*****************************************
# Préparation des librairie et des outils
# pour le nettoyage des tweets
#*****************************************

# Expressions régulières
import re

# Pontuations
import string
ponctuations = list(string.punctuation)
print(ponctuations)

# Tokénisation
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Lemmatisation
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()

# Charger les stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
mots_vides = stopwords.words('english')
print('\n')
print(mots_vides)

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'wh

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JK253\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\JK253\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JK253\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Nettoyer le contenu des tweets

In [8]:
# fonction pour nettoyage de chaque document
# tweet = corps du tweet = document
# ponctuations : liste des ponctuations
# stopwords : liste des stopwords à retirer
# lem : fonction pour la lemmatisation des termes

def clean_tweet(tweet, ponctuations, stopwords, lem):
    # Harmonisation de la casse
    temp = tweet.lower()
    # retier les contractions en anglais
    temp = re.sub("'", "", temp)
    # retrait des @
    temp = re.sub("@[A-Za-z0-9_]+", "", temp)
    # retrait des #
    temp = re.sub("#[A-Za-z0-9_]+", "", temp)
    # retrait des liens web (http et https)
    temp = re.sub(r'http\S+','', temp)
    # retrait des ponctuations
    temp = "".join([char for char in list(temp) if not (char in ponctuations)])
    # retrait des nombres
    temp = re.sub("[0-9]", "", temp)
    # tokénisation
    temp = word_tokenize(temp)
    # lemmatisation des termes
    temp = [lem.lemmatize(mot) for mot in temp]
    # retrait des stopwords
    temp = [mot for mot in temp if not mot in stopwords]
    # retirer les tokens de moins de 3 caractères
    temp = [mot for mot in temp if len(mot) >= 3]
    return temp

In [9]:
# Appliquer le nettoyage au corpus
corpus = list(data.tweet)
corpus = [clean_tweet(doc, ponctuations, mots_vides, lem) for doc in corpus]

In [10]:
corpus[0]

['school', 'grrr', 'mood', 'take', 'make', 'back', 'put', 'teacher', 'leave']

In [11]:
# Nouvelle dataframe
df_1 = pd.DataFrame({'label': data.polarite ,'tweet': corpus})
print("avant:.........")
data.head(10)

avant:.........


,label,date_du_poste,utilisateur,tweet,polarite
0,0,Mon Apr 20 00:27:23 PDT 2009,TapeLaLaLa,", school! GrRr im not in the mood to take my ...",negative
1,0,Fri May 29 13:29:39 PDT 2009,Billie_Anne,@pfchangs I can't direct message you for my fr...,negative
2,0,Sat May 30 04:39:55 PDT 2009,rascal2pt0,Still not skinny. Up 0.4 lbs this morning,negative
3,0,Fri Jun 05 23:01:03 PDT 2009,Amaryste78,@PattinsonRobT Well some of us just want to sh...,negative
4,0,Sat Jun 06 11:30:11 PDT 2009,Alovepoet,extra credit for history is golden! scared abo...,negative
5,0,Sun May 31 18:31:36 PDT 2009,TataJones,"@tommcfly back to Rio tomorrow,please. I need ...",negative
6,0,Sat Apr 18 07:15:28 PDT 2009,Ant_Cashanova,Slight hangover,negative
7,0,Wed Jun 17 12:16:33 PDT 2009,Bolanile,"I must admit, I am jealous of the 88 degree we...",negative
8,0,Sat Jun 20 04:32:00 PDT 2009,overhope,"@nickvampie sorry bro, after that evil one th...",negative
9,0,Tue Jun 16 18:27:54 PDT 2009,Amy_Pearson1993,@shaundiviney Shaun I have a problem :S wens I...,negative


In [12]:
print("après:.........")
df_1.head(10)

après:.........


,label,tweet
0,negative,"[school, grrr, mood, take, make, back, put, te..."
1,negative,"[cant, direct, message, free, lunch]"
2,negative,"[still, skinny, morning]"
3,negative,"[well, want, share, sorry, hacked]"
4,negative,"[extra, credit, history, golden, scared, algre..."
5,negative,"[back, rio, tomorrowplease, need, youi, need, ..."
6,negative,"[slight, hangover]"
7,negative,"[must, admit, jealous, degree, weather, today,..."
8,negative,"[sorry, bro, evil, one, ran, longer, back]"
9,negative,"[shaun, problem, wen, preordered, album, put, ..."


In [13]:
# des tweets vides après nettoyage?
print(df_1.loc[df_1.tweet==""].shape[0])

0


In [14]:
# retrait des tweets correspondants
df_ok = df_1.loc[df_1.tweet != ""]
print(df_ok.shape)

(40000, 2)


In [15]:
df_ok.head()

,label,tweet
0,negative,"[school, grrr, mood, take, make, back, put, te..."
1,negative,"[cant, direct, message, free, lunch]"
2,negative,"[still, skinny, morning]"
3,negative,"[well, want, share, sorry, hacked]"
4,negative,"[extra, credit, history, golden, scared, algre..."


## Charger une représentation pré-entraînée des termes

In [16]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models.keyedvectors import KeyedVectors

In [18]:
path = '../P7/'

In [19]:
glove2word2vec(glove_input_file=path+"glove.twitter.27B.100d.txt", word2vec_output_file="gensim_glove_vectors.txt")

(1193514, 100)

In [20]:
glove_model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)

A c ette instant j'ai utilisé glove2word2vec pour convertir des vecteurs GloVe au format texte au format texte word2vec

In [21]:
#dimension
print(glove_model.vectors.shape)

(1193514, 100)


In [23]:
#coordonnées de frog(grenouille)
print(glove_model['frog'])

[-2.0432e-01 -2.6150e-01 -1.6633e-01  4.6853e-01 -4.5815e-01  2.7206e-01
  7.2837e-01 -2.7332e-01  7.1900e-03  1.3625e-02  1.2160e-01  5.8155e-01
 -1.9450e+00  7.3037e-02  3.2283e-01  7.7129e-01  4.1995e-01  3.6865e-01
  1.3315e-01 -5.6905e-01  1.2460e-01 -4.2542e-01  2.1237e-01 -4.0740e-01
  1.2356e+00  5.4340e-01 -2.2625e-01  3.0468e-02 -4.6312e-02  3.7316e-01
  5.9658e-01  2.9139e-01  5.0718e-01 -2.9507e-01 -8.5241e-01  2.3822e-01
  4.6839e-01 -1.8568e-01  1.7349e-01 -6.8062e-02  3.4812e-01 -1.8539e-01
 -1.3700e-01  2.5837e-01  7.0864e-01  8.3642e-01 -2.3989e-02  9.6164e-01
  4.9107e-01  1.6798e-01  9.6609e-01 -2.8883e-01  3.8461e-01 -3.8119e-01
  1.5834e-03 -1.1867e-01  2.7806e-01 -1.2263e-01  9.4348e-02  4.0990e-01
 -7.7709e-01 -2.3403e-01  1.5025e-01 -1.3776e-01 -7.2865e-01 -1.2810e-01
 -3.3601e-01  2.8861e-01 -2.6464e-01 -5.6897e-02  7.8214e-01 -4.2059e-01
 -2.2993e-01 -9.2306e-01 -7.5719e-02  8.3283e-02 -2.5813e-01  4.9935e-01
  8.3127e-01 -3.5099e-01  1.7317e+00  6.3388e-01 -6

In [25]:
#similarité avec frog(grenouille)
print(glove_model.most_similar(['frog']))

[('squirrel', 0.7796489596366882), ('toad', 0.7313696146011353), ('turtle', 0.7177964448928833), ('rabbit', 0.7150670289993286), ('monkey', 0.6988509893417358), ('frogs', 0.6952397227287292), ('pig', 0.6924378871917725), ('unicorn', 0.6852490901947021), ('elephant', 0.6847099661827087), ('cow', 0.6808868646621704)]


In [26]:
#l'exemple emblématique 
print(glove_model.most_similar(positive=['king','woman'],negative=['man']))

[('queen', 0.7052315473556519), ('prince', 0.6666139364242554), ('mother', 0.6436765193939209), ('royal', 0.6417251229286194), ('father', 0.5952690243721008), ('african', 0.5883978009223938), ('princess', 0.5882176160812378), ('called', 0.5842776894569397), ('meets', 0.5840279459953308), ('american', 0.5815179944038391)]


## Coordonnées des documents à partir des termes

Représentation d'un document : moyenne des vecteurs des termes qui le composent.

In [27]:
#librairie numpy
import numpy

#fonction pour transformer un document en vecteur
#à partir des tokens qui le composent
#entrée : doc à traiter
#         modèle préentrainé
#sortie : vecteur représentant le document
def my_doc_2_vec(doc,glove_model):
    #dimension de représentation
    p = glove_model.vectors.shape[1]
    #initialiser le vecteur
    vec = numpy.zeros(p)
    #nombre de tokens trouvés
    nb = 0
    #traitement de chaque token du document
    for tk in doc:
        #ne traiter que les tokens reconnus
        try:
            values = glove_model[tk]
            vec = vec + values
            nb = nb + 1.0
        except:
            pass
    #faire la moyenne des valeurs
    #uniquement si on a trové des tokens reconnus bien sûr
    if (nb > 0.0):
        vec = vec/nb
    #renvoyer le vecteur
    #si aucun token trouvé, on a un vecteur de valeurs nulles
    return vec

In [28]:
#traiter les documents du corpus corpus
docsVec = list()
#pour chaque document du corpus nettoyé
for doc in df_ok['tweet']:
    #calcul de son vecteur
    vec = my_doc_2_vec(doc,glove_model)
    #ajouter dans la liste
    docsVec.append(vec)
#transformer en matrice numpy
matVec = numpy.array(docsVec)
print(matVec.shape)

(40000, 100)


In [29]:
#transformer en data frame
df = pd.DataFrame(matVec,columns=["v"+str(i+1) for i in range(matVec.shape[1])])
df.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100
0,0.060899,0.198926,0.300655,0.129622,0.153767,0.075731,0.502758,0.147597,-0.013224,0.275639,...,-0.121738,0.097694,0.068696,0.199511,-0.059477,-0.123621,-0.122238,0.067667,-0.064020,0.135521
1,0.081094,0.560460,-0.498200,0.070908,-0.185006,-0.251941,0.469116,0.095297,0.361210,0.265204,...,-0.239547,0.203879,-0.086067,0.040713,0.193605,0.304194,-0.134565,0.035534,-0.293732,-0.070785
2,-0.065943,-0.301713,0.122880,0.082153,-0.053890,0.291073,0.279334,0.174660,-0.118000,0.184008,...,-0.023398,0.132623,0.312067,0.094859,-0.150200,-0.014259,-0.206707,0.371103,0.253408,-0.180955
3,0.281499,0.637894,-0.124118,0.160039,-0.456024,-0.284932,0.430783,-0.068897,0.594408,0.392441,...,-0.172953,0.034142,-0.032593,0.004753,0.047503,-0.360497,-0.282181,0.101678,0.278702,0.155895
4,-0.218933,0.314345,0.312355,0.019018,-0.149179,-0.114710,0.281285,0.022573,0.071846,0.219789,...,-0.056736,-0.128943,-0.070469,-0.230003,0.256608,0.283126,0.009690,0.313177,-0.004764,0.151090


In [30]:
#ajouter la classe
df['label'] = df_ok.label
df.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v92,v93,v94,v95,v96,v97,v98,v99,v100,label
0,0.060899,0.198926,0.300655,0.129622,0.153767,0.075731,0.502758,0.147597,-0.013224,0.275639,...,0.097694,0.068696,0.199511,-0.059477,-0.123621,-0.122238,0.067667,-0.064020,0.135521,negative
1,0.081094,0.560460,-0.498200,0.070908,-0.185006,-0.251941,0.469116,0.095297,0.361210,0.265204,...,0.203879,-0.086067,0.040713,0.193605,0.304194,-0.134565,0.035534,-0.293732,-0.070785,negative
2,-0.065943,-0.301713,0.122880,0.082153,-0.053890,0.291073,0.279334,0.174660,-0.118000,0.184008,...,0.132623,0.312067,0.094859,-0.150200,-0.014259,-0.206707,0.371103,0.253408,-0.180955,negative
3,0.281499,0.637894,-0.124118,0.160039,-0.456024,-0.284932,0.430783,-0.068897,0.594408,0.392441,...,0.034142,-0.032593,0.004753,0.047503,-0.360497,-0.282181,0.101678,0.278702,0.155895,negative
4,-0.218933,0.314345,0.312355,0.019018,-0.149179,-0.114710,0.281285,0.022573,0.071846,0.219789,...,-0.128943,-0.070469,-0.230003,0.256608,0.283126,0.009690,0.313177,-0.004764,0.151090,negative


## Prédiction et évaluation en test

In [31]:
#partition apprentissage test
from sklearn.model_selection import train_test_split
dfTrain, dfTest = train_test_split(df,train_size=0.7,stratify=df.label,random_state=0)
print(dfTrain.shape)
print(dfTest.shape)

(28000, 101)
(12000, 101)


Je mets un échantillonage stratifié pour avoir les mêmes proportion de classe

In [32]:
#SVM avec un noyau RBF par défaut
from sklearn.svm import SVC
clf = SVC(random_state=0)
clf.fit(dfTrain[dfTrain.columns[:-1]],dfTrain.label)

SVC(random_state=0)

In [33]:
#prédiction en test
pred = clf.predict(dfTest[dfTest.columns[:-1]])
print(pred.shape)

(12000,)


In [34]:
#évaluation des performances
from sklearn import metrics
print(metrics.classification_report(dfTest.label,pred))

              precision    recall  f1-score   support

    negative       0.74      0.74      0.74      6000
    positive       0.74      0.75      0.74      6000

    accuracy                           0.74     12000
   macro avg       0.74      0.74      0.74     12000
weighted avg       0.74      0.74      0.74     12000



## Déploiement sur un document supplémentaire

In [35]:
#document à traiter
blog_avion = "My next plane ticket has just been booked! I fly in March to the Philippines. I am currently in the process of preparing for my next trip: what are the places I would like to discover, which itinerary to choose and in which cities to sleep, which domestic flights I will book to move between the islands etc. A trip by plane is an action that is anticipated and prepared a minimum. We can also face hazards, and it is important to know how to react. I indicate in this article various and varied tips on the theme of air travel. During the flight, the time may seem long, especially depending on the duration of your trip. Take something to take care of, and what not to be embarrassed (Quies balls can be useful). Also, feel free to get up from time to time if you need to stretch your legs. Be aware that the toilet is unavailable during take-off and landing, take precautions. A tip, always carry a photocopy of your passport, it can help in case of loss or theft that would occur during the trip. We also often talk about flight delay or cancellation. The wait seems endless in these cases, and travelers too often lack information. It is therefore necessary to try to take his evil in patience. Delayed plane, cancelled flight,companies like Indemnflight can help you get compensated. Note that the delay must be more than 3 hours for compensation to be possible. The advantage of going through Indemnflight is that experts take care of the administrative procedures for you, and they know the rights of passengers well. Note that they are paid only when the compensation is validated. You therefore delegate the management of the request, and then pay them in return a part of the compensation. I personally have only 2h30 maximum of delay to my credit, and have never tested the claim. Finally, the loss of luggage is also a thorny subject! If you do not find it at the exit of the plane, go to a counter available at the airport to report it and find a solution. The price of a plane ticket varies a lot depending on many factors. The level of demand, the time of year, the day and time of booking and the booking time before the departure date are very important elements to take into account if you have in mind to make a good deal when booking your flight. To learn more, I advise you to read this article from my travel blog: How to find a cheap plane ticket? Follow these 11 tips! When booking your flight, check that the formalities of entry into the desired country are feasible on time (example: a visa application must be made in advance), and that your identity card and/or passport will still be valid. Note that for many destinations, the passport must often be valid for a period of 6 months from the date of arrival. It is advisable to book flights well in advance. You rarely get a good deal when you buy a plane ticket a few days before departure. And in this case we take the risk of no longer having a place. In the case of my next trip to the Philippines, it is for these reasons that I will soon book the 2 domestic flights. Especially since I am going to travel with the national airline, and I have the impression that the weight of the luggage allowed is less than the 23 kg allowed on my international flight. I must therefore take this into account. Finally, it is necessary to pay attention to the information provided during the reservation. An error on the last name, or a time on the dates can be expensive, there is in this case a risk of paying extra to make this type of change, or even having to cancel your trip. First of all, find out a little in advance about how you will reach the airport (car + parking, bus, train...). When packing your suitcase, try to think of everything you will need on the spot. Also be aware that some products are prohibited in cabin baggage (liquid products greater than 100ml, knives etc.). Check the airline's website for an exhaustive list, and avoid having certain products confiscated. It is advisable to arrive 1h30 to 2h in advance for short or medium haul flight. 2h to 3h are recommended for international flights. This delay varies according to several elements: is your plane ticket ready or are you going to do the formalities at the airport? Do you have luggage to drop off at the counter? How big is the airport and in which area will you board? To help you and go further, discover the guide: practical tips for flying the first time: the steps to follow at the airport. Do you have any other tips to share when flying? Or adventures to share? Feel free to indicate them in the comments."

In [36]:
# nettoyage
my_clean = clean_tweet(blog_avion, ponctuations, mots_vides,lem)
print(my_clean)

['next', 'plane', 'ticket', 'booked', 'fly', 'march', 'philippine', 'currently', 'process', 'preparing', 'next', 'trip', 'place', 'would', 'like', 'discover', 'itinerary', 'choose', 'city', 'sleep', 'domestic', 'flight', 'book', 'move', 'island', 'etc', 'trip', 'plane', 'action', 'anticipated', 'prepared', 'minimum', 'also', 'face', 'hazard', 'important', 'know', 'react', 'indicate', 'article', 'various', 'varied', 'tip', 'theme', 'air', 'travel', 'flight', 'time', 'may', 'seem', 'long', 'especially', 'depending', 'duration', 'trip', 'take', 'something', 'take', 'care', 'embarrassed', 'quies', 'ball', 'useful', 'also', 'feel', 'free', 'get', 'time', 'time', 'need', 'stretch', 'leg', 'aware', 'toilet', 'unavailable', 'takeoff', 'landing', 'take', 'precaution', 'tip', 'always', 'carry', 'photocopy', 'passport', 'help', 'case', 'loss', 'theft', 'would', 'occur', 'trip', 'also', 'often', 'talk', 'flight', 'delay', 'cancellation', 'wait', 'seems', 'endless', 'case', 'traveler', 'often', 'la

### Transformation du document en vecteur et prédiction

In [37]:
#avec le modèle pré-entrainé
#et avec la fonction ci-dessus
my_vec = my_doc_2_vec(my_clean,glove_model)
print(my_vec)

[ 7.42997222e-02  1.16145684e-01  3.87272052e-02 -1.20921594e-01
  1.08192906e-01  4.18694296e-02  3.35730307e-01 -1.17281475e-01
  6.41943145e-02  3.38872367e-02  2.64979805e-02 -1.73002679e-01
 -3.18489433e+00  9.33958689e-02  2.81091918e-02 -6.14780135e-02
  3.22580343e-02  1.22408663e-01 -8.05992555e-02 -2.37422325e-01
 -8.49119562e-02 -1.36923613e-01 -1.93114940e-02  3.23095252e-02
 -1.43564945e-01  2.80699916e-01  1.00997381e-01  9.97032886e-02
  4.53340388e-02  7.60205715e-02  4.45375788e-02  5.21420077e-02
 -2.46802528e-01 -2.34085313e-02  2.36621924e-01  4.01339422e-02
  1.23273508e-01  6.53882959e-02  1.12388811e-01  2.54380495e-02
 -2.53524606e-01  1.24888294e-01  1.72107432e-01 -8.64640767e-02
  3.44709201e-01 -3.41666089e-02  8.14689056e-02 -1.96896431e-01
 -6.96362389e-02  3.10120713e-01  1.52974121e-01  8.52233496e-02
 -2.34912676e-01  1.09456171e-01 -1.24663969e-01  1.82868192e-01
  1.75005275e-02 -6.22283272e-02 -8.58723371e-02 -1.27122005e-02
  2.45708276e-01  1.89394

In [38]:
#prédiction avec le SVM
pred_my_doc = clf.predict(my_vec.reshape(1,-1))

#le commentaire est de nature positive
print(pred_my_doc)

['negative']


## Sauvegarder le modèle Glove

In [39]:
import joblib
# maintenant je l’enregistre dans un fichier
joblib.dump(clf, 'P7_01_modèlesurmesure_04_embedding_Glove.pkl')

['P7_01_modèlesurmesure_04_embedding_Glove.pkl']